In [ ]:
# 学習した重みから画像を生成するサンプルコード

In [1]:
# GPU情報の確認
!nvidia-smi

Fri Feb  5 11:13:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# google driveからcolaboにファイルをコピーするメソッド
import requests

def download_file_from_google_drive(id, destination):
    print('---- download_file_from_google_drive start ----')
    URL = "https://docs.google.com/uc?export=download"

    try:
        session = requests.Session()

        response = session.get(URL, params = { 'id' : id }, stream = True)
        print('response.status_code: {}'.format(response.status_code))
        print('response.headers: {}'.format(response.headers))

        token = get_confirm_token(response)
        print('token'.format(token))

        if token:
            print(token)
            params = { 'id' : id, 'confirm' : token }
            response = session.get(URL, params = params, stream = True)

        save_response_content(response, destination)
    except Exception as e:
        print(e)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        print('key: {}'.format(key))
        print('value: {}'.format(value))
       
        if key.startswith('download_warning'):
            return value
            
        if key == 'NID':
            print('NID')
            token = value.split('=')
            print(token)
            return token[1]

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [3]:
# 実行環境のrootディレクトリの確認
%cd /content
!ls -lah

/content
total 16K
drwxr-xr-x 1 root root 4.0K Feb  1 17:27 .
drwxr-xr-x 1 root root 4.0K Feb  5 11:09 ..
drwxr-xr-x 1 root root 4.0K Feb  3 15:19 .config
drwxr-xr-x 1 root root 4.0K Feb  1 17:27 sample_data


In [4]:
# StyleGAN2のプログラムをダウンロード。
# ① Githubからプログラムをダウンロードし、zipで固めてGoogle Driveにアップロードします。
# ② ファイルの共有設定を「リンクを知っている全員」に変更します
# ③ 上記のファイルのファイルIDを下記のfile_idに指定します。
# 直ぐに実行したい方はこのままのファイルIDで実行してください。
# https://drive.google.com/file/d/1wH9mulsQ0RtpgF4jdpuBSytgl23FlYha/view?usp=sharing
file_id = '1wH9mulsQ0RtpgF4jdpuBSytgl23FlYha'
destination = './stylegan2_pytorch.zip'
download_file_from_google_drive(file_id, destination)

---- download_file_from_google_drive start ----
response.status_code: 200
response.headers: {'X-GUploader-UploadID': 'ABg5-Uwpwu44cF9atbWZd07MBBOwHhWVIsAfrVgFjaP4lZ_qeqBtxf1YTDncG8-7CWIgVH_LIcRqs9T-Mk5Md7GFL58', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'false', 'Access-Control-Allow-Headers': 'Accept, Accept-Language, Authorization, Cache-Control, Content-Disposition, Content-Encoding, Content-Language, Content-Length, Content-MD5, Content-Range, Content-Type, Date, X-Goog-Sn-Metadata, X-Goog-Sn-PatientId, GData-Version, google-cloud-resource-prefix, x-goog-request-params, Host, If-Match, If-Modified-Since, If-None-Match, If-Unmodified-Since, Origin, OriginToken, Pragma, Range, Slug, Transfer-Encoding, hotrod-board-name, hotrod-chrome-cpu-model, hotrod-chrome-processors, Want-Digest, x-chrome-connected, X-ClientDetails, X-Client-Version, X-Firebase-Locale, X-Goog-Firebase-Installations-Auth, X-Firebase-Client, X-Firebase-Client-Log-Type, X-Firebase-GMPID,

In [5]:
# 実行環境のrootディレクトリに配置されたプログラムのzipファイルを展開します。
!unzip stylegan2_pytorch.zip

Archive:  stylegan2_pytorch.zip
   creating: stylegan2_pytorch/
  inflating: stylegan2_pytorch/synthesis_network.py  
  inflating: stylegan2_pytorch/simple_upfirdn_2d.py  
  inflating: stylegan2_pytorch/downsample_conv_2d.py  
  inflating: stylegan2_pytorch/.DS_Store  
  inflating: __MACOSX/stylegan2_pytorch/._.DS_Store  
  inflating: stylegan2_pytorch/tensorboard_logger.py  
  inflating: stylegan2_pytorch/generator_loss.py  
  inflating: stylegan2_pytorch/block_discreminator.py  
  inflating: stylegan2_pytorch/base_layer.py  
  inflating: stylegan2_pytorch/utility.py  
  inflating: stylegan2_pytorch/frechet_Inception_distance.py  
  inflating: stylegan2_pytorch/training_back.py  
  inflating: stylegan2_pytorch/fused_bias_activation.py  
  inflating: stylegan2_pytorch/inception.py  
  inflating: stylegan2_pytorch/to_rgb.py  
  inflating: stylegan2_pytorch/minibatch_stddev_layer.py  
  inflating: stylegan2_pytorch/datasource.py  
  inflating: stylegan2_pytorch/dataset.py  
  inflating: 

In [6]:
# Google driveのマウントを行います。
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# マウントしたGoogle Drive情報の確認
!ls drive -lah

total 16K
dr-x------ 2 root root 4.0K Feb  5 11:15 .file-revisions-by-id
drwx------ 4 root root 4.0K Feb  5 11:15 MyDrive
dr-x------ 2 root root 4.0K Feb  5 11:15 .shortcut-targets-by-id
drwx------ 2 root root 4.0K Feb  5 11:15 .Trash


In [29]:
# 画像の生成を行います
# --batch_size: 学習を行った際のバッチサイズを指定します
# --resolution: 学習を行った際の画像の解像度を指定します
# --model_path: 学習済みモデルを配置したディレクトリを指定します。
# 学習済みモデルは下記よりダウンロードすることもできます。
# https://drive.google.com/file/d/146Gmqh_PFf8AVXKij1yWCj1xHfG4urPo/view?usp=sharing
# --output_path: 生成した画像を格納するディレクトリを指定します。
# --seeds: 標準正規分布を作成する際の乱数のシードを指定します。
# seedsに指定した個数分、画像が生成され、--output_pathで指定したディレクトリ格納されます。
# --truncation-psi: 0 〜 1の間を指定します。0に近いと標準的な画像が生成され、1.0に近い値を指定すると、標準から外れた画像が生成されます。
# 0に近いと品質は良いですがつまらない画像が、1.0に近いと変わった画像が生成されますが、品質は悪くなります。
# --latent_dim: 中間潜在変数の次元数を指定します。学習時と同じ値を設定します。

%cd /content/stylegan2_pytorch
!python generate_images.py \
--batch_size=4 \
--resolution=512 \
--model_path=../drive/'My Drive'/project/gan_book/model \
--output_path=../drive/'My Drive'/project/gan_book/results \
--seeds=1,13,113,221,1059,3567,6691,10336,19440,65531 \
--truncation-psi=0.3 \
--latent_dim=512

/content/stylegan2_pytorch
Namespace(batch_size=4, latent_dim=512, model_path='../drive/My Drive/project/gan_book/model', output_path='../drive/My Drive/project/gan_book/results', resolution=512, seeds=[1, 13, 113, 221, 1059, 3567, 6691, 10336, 19440, 65531], truncation_psi=0.3)
2021-02-05 11:58:18.029322: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
